In [1]:
import polars as pl
import dotenv
import os
from libraries.client_stashapp import get_stashapp_client
from libraries.StashDbClient import StashDbClient

# Format a StashDB ID for use as an aliasin Stash
stashdb_id_alias_prefix = "StashDB ID: "
def format_stashdb_id(id):
    return f"{stashdb_id_alias_prefix}{id}"

def contains_cjk(text):
    """Check if text contains CJK (Chinese, Japanese, Korean) characters."""
    # Unicode ranges for CJK characters
    cjk_ranges = [
        (0x4E00, 0x9FFF),   # CJK Unified Ideographs
        (0x3040, 0x309F),   # Hiragana
        (0x30A0, 0x30FF),   # Katakana
        (0x3400, 0x4DBF),   # CJK Unified Ideographs Extension A
        (0xF900, 0xFAFF),   # CJK Compatibility Ideographs
        (0xAC00, 0xD7AF),   # Korean Hangul Syllables
    ]
    
    return any(any(ord(char) >= start and ord(char) <= end 
               for start, end in cjk_ranges) 
               for char in text)


dotenv.load_dotenv()

stash = get_stashapp_client()

stashbox_client = StashDbClient(
    os.getenv("STASHDB_ENDPOINT"),
    os.getenv("STASHDB_API_KEY"),
)

dUsing stash (v0.27.2-37-g0621d871) endpoint at http://localhost:6969/graphql
dPersisting Connection to Stash with ApiKey...


# Merging tags


In [166]:
to_be_merged_tag = stash.find_tag({ "name": "Standing Double Penetration" })
target_tag = stash.find_tag({ "name": "Standing Sex (DP)" })

print(to_be_merged_tag)
print("=>")
print(target_tag)
print()

scenes = stash.find_scenes({ "tags": { "value": [to_be_merged_tag['id']], "modifier": "INCLUDES" }}, fragment="id title tags { id name }")
galleries = stash.find_galleries({ "tags": { "value": [to_be_merged_tag['id']], "modifier": "INCLUDES" }}, fragment="id title tags { id name }")
images = stash.find_images({ "tags": { "value": [to_be_merged_tag['id']], "modifier": "INCLUDES" }}, fragment="id title tags { id name }")
markers = stash.find_scene_markers({ "tags": { "value": [to_be_merged_tag['id']], "modifier": "INCLUDES" }}, fragment="id scene { id title } title primary_tag { id name } tags { id name }")

print(f"Scenes: {len(scenes)}")
print(f"Markers: {len(markers)}")
print(f"Galleries: {len(galleries)}")
print(f"Images: {len(images)}")


{'id': '7178', 'name': 'Standing Double Penetration', 'description': 'A female performer having vaginal and anal penetrative sex simultaneously while standing up', 'aliases': ['DP - Standing', 'Standing DP', 'Standing.Double.Penetration'], 'ignore_auto_tag': False, 'created_at': '2024-04-23T12:52:25Z', 'updated_at': '2024-09-19T07:58:21+03:00', 'favorite': False, 'image_path': 'http://localhost:6969/tag/7178/image?t=1726721901&default=true', 'scene_count': 6, 'scene_marker_count': 2, 'image_count': 0, 'gallery_count': 0, 'performer_count': 0, 'studio_count': 0, 'group_count': 0, 'parents': [{'id': '7753'}], 'children': [], 'parent_count': 1, 'child_count': 0}
=>
{'id': '7993', 'name': 'Standing Sex (DP)', 'description': '', 'aliases': ['StashDB ID: 01eb4084-62ac-498c-9115-43fd709c23ca'], 'ignore_auto_tag': False, 'created_at': '2025-01-26T17:44:27+02:00', 'updated_at': '2025-01-26T17:44:27+02:00', 'favorite': False, 'image_path': 'http://localhost:6969/tag/7993/image?t=1737906267&defau

In [167]:
# Update scenes
for scene in scenes:
    scene_id = scene['id']
    current_scene_tag_ids = [tag['id'] for tag in scene['tags']]
    update_scene_tag_ids = [tag_id for tag_id in current_scene_tag_ids if tag_id != to_be_merged_tag['id']] + [target_tag['id']]
    stash.update_scene({ "id": scene_id, "tag_ids": update_scene_tag_ids })
    print(f"Updated scene {scene_id} with tag {target_tag['name']}")

Updated scene 6969 with tag Standing Sex (DP)
Updated scene 10995 with tag Standing Sex (DP)
Updated scene 11069 with tag Standing Sex (DP)
Updated scene 13778 with tag Standing Sex (DP)
Updated scene 2770 with tag Standing Sex (DP)
Updated scene 11742 with tag Standing Sex (DP)


In [168]:
# Update markers
for marker in markers:
    marker_id = marker['id']
    current_marker_tag_ids = [tag['id'] for tag in marker['tags']]
    update_marker_tag_ids = [tag_id for tag_id in current_marker_tag_ids if tag_id != to_be_merged_tag['id']] + [target_tag['id']]
    stash.update_scene_marker({ "id": marker_id, "title": target_tag["name"], "primary_tag_id": target_tag['id'] })
    print(f"Updated marker {marker_id} with tag {target_tag['name']} for scene {marker['scene']['title']} (ID: {marker['scene']['id']})")


Updated marker 62786 with tag Standing Sex (DP) for scene The Dinner Party (ID: 11069)
Updated marker 61698 with tag Standing Sex (DP) for scene ZZ Confidential (ID: 10995)


In [38]:
# Update galleries
for gallery in galleries:
    gallery_id = gallery['id']
    current_gallery_tag_ids = [tag['id'] for tag in gallery['tags']]
    update_gallery_tag_ids = [tag_id for tag_id in current_gallery_tag_ids if tag_id != to_be_merged_tag['id']] + [target_tag['id']]
    stash.update_gallery({ "id": gallery_id, "tag_ids": update_gallery_tag_ids })
    print(f"Updated gallery {gallery_id} with tag {target_tag['name']}")


In [13]:
# Update images
for image in images:
    image_id = image['id']
    current_image_tag_ids = [tag['id'] for tag in image['tags']]
    update_image_tag_ids = [tag_id for tag_id in current_image_tag_ids if tag_id != to_be_merged_tag['id']] + [target_tag['id']]
    stash.update_image({ "id": image_id, "tag_ids": update_image_tag_ids })
    print(f"Updated image {image_id} with tag {target_tag['name']}")

In [169]:
stash.destroy_tag(to_be_merged_tag['id'])


# Syncing tags from StashDB to Stash

In [83]:
stashdb_tags = stashbox_client.query_tags()

In [84]:
# Get tags from StashDB
df_stashdb_tags = pl.DataFrame(stashdb_tags)

df_stashdb_tags = df_stashdb_tags.with_columns(
    pl.col("category").map_elements(lambda x: x['id'] if x else None, return_dtype=pl.Utf8).alias("category_id"),
    pl.col("category").map_elements(lambda x: x['name'] if x else None, return_dtype=pl.Utf8).alias("category_name"),
    pl.col("category").map_elements(lambda x: x['description'] if x else None, return_dtype=pl.Utf8).alias("category_description"),
    pl.col("category").map_elements(lambda x: x['group'] if x else None, return_dtype=pl.Utf8).alias("category_group"),
).drop("category")

df_stashdb_tags


id,name,description,aliases,deleted,created,updated,category_id,category_name,category_description,category_group
str,str,str,list[str],bool,str,str,str,str,str,str
"""9441c3ad-41d2-4d6e-bc97-54ad8c…","""120 FPS""","""Scenes offered at 120 frames p…","[""120帧""]",false,"""2022-04-05T20:28:06Z""","""2024-02-17T18:36:12.991842Z""","""ef4ae6d1-d13c-4195-b47e-f245e4…","""Shot Type""","""Technical details of how a vid…","""SCENE"""
"""42d9e5c4-1a1d-4c93-bf47-9086f2…","""12K Available""","""Scenes offered in a resolution…","[""12K"", ""12K Shemale VR Porn"", … ""True 12K""]",false,"""2024-12-03T05:31:48.278753Z""","""2024-12-03T05:31:48.278753Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE"""
"""8534d108-1f4c-42f9-8caa-5ca906…","""18+""","""Primary performer (not charact…","[""18 Plus"", ""Over 18""]",false,"""2024-03-30T04:09:32.347616Z""","""2024-03-30T04:09:32.347616Z""","""b40e08dd-314e-40ca-8fdb-bf7541…","""Age Group""","""Implied age ranges for charact…","""PEOPLE"""
"""103a1f16-83e1-4b9f-ab14-e85e04…","""180°""","""Virtual reality scenes with a …","[""180"", ""180 FOV"", … ""VR180""]",false,"""2020-04-27T18:59:52Z""","""2023-05-25T09:25:21.314083Z""","""ef4ae6d1-d13c-4195-b47e-f245e4…","""Shot Type""","""Technical details of how a vid…","""SCENE"""
"""6cd87d98-eea8-4b97-9db9-aa38a9…","""1800s""","""Inspired by the history and cu…","[""1800's"", ""19th Century"", … ""Victorian""]",false,"""2024-02-15T10:25:01.839985Z""","""2024-02-15T10:25:01.839985Z""","""0319d5d6-a07f-4e0d-809d-c09fb1…","""Themes""","""Events, contexts, or fetishes …","""SCENE"""
…,…,…,…,…,…,…,…,…,…,…
"""e7f1f848-4350-4bda-925c-b01235…","""Young Man (22–30)""","""Male presented as generally yo…","[""Young Guy"", ""Young Male"", … ""青年男子 (22–30)""]",false,"""2020-04-27T18:59:52Z""","""2024-11-12T06:27:11.374593Z""","""b40e08dd-314e-40ca-8fdb-bf7541…","""Age Group""","""Implied age ranges for charact…","""PEOPLE"""
"""84ba8ef1-084c-46f8-b352-31154f…","""Young Woman (22–30)""","""Female character presented as …","[""Chick"", ""Woman (20-29)"", … ""Youthful Woman""]",false,"""2020-04-27T18:59:52Z""","""2024-11-14T00:15:52.456833Z""","""b40e08dd-314e-40ca-8fdb-bf7541…","""Age Group""","""Implied age ranges for charact…","""PEOPLE"""
"""6c0a2824-acd2-4b64-9a2c-634bd9…","""Zentai""","""Skin-tight garment that covers…","[""Zentai Suit""]",false,"""2022-07-10T22:17:47.537338Z""","""2022-07-10T22:17:47.537338Z""","""dc566ccc-0584-41d8-b9f5-4d8680…","""Clothing""","""Articles or styles of clothing…","""PEOPLE"""


In [5]:
df_stashdb_tags.write_json("H:\\Parquet Data\\StashDB\\stashdb_tags.json")

In [199]:
# Get tags from Stash
stash_tags = stash.find_tags()
df_stash_tags = pl.DataFrame(stash_tags)
df_stash_tags = df_stash_tags.with_columns(
    pl.col("aliases").map_elements(
        lambda aliases: next(
            (alias[len(stashdb_id_alias_prefix):] for alias in aliases if isinstance(alias, str) and alias.startswith(stashdb_id_alias_prefix)),
            None
        ),
        return_dtype=pl.Utf8
    ).alias("stashdb_id")
)
df_stash_tags

id,name,description,aliases,ignore_auto_tag,created_at,updated_at,favorite,image_path,scene_count,scene_marker_count,image_count,gallery_count,performer_count,studio_count,group_count,parents,children,parent_count,child_count,stashdb_id
str,str,str,list[str],bool,str,str,bool,str,i64,i64,i64,i64,i64,i64,i64,list[struct[1]],list[struct[1]],i64,i64,str
"""5045""","""2D Available""","""3D or VR scenes that offer a m…","[""2-D"", ""2D"", … ""Two-Dimensional""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5045…",1,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""1257be8b-d1ec-4cb1-bb22-beeb89…"
"""5049""","""3D Available""","""Offered in a format with a thr…","[""3-D"", ""3D"", … ""Three-Dimensional""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5049…",50,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""52992c2c-4617-4540-8ca4-291e9c…"
"""5050""","""3K Available""","""Scenes offered in a resolution…","[""1600p"", ""3K VP9"", … ""StashDB ID: c3794d99-1b5b-47b3-86f7-75ff2de748b8""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5050…",7,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""c3794d99-1b5b-47b3-86f7-75ff2d…"
"""5051""","""3rd Person Narrative""","""Features a storyline with fict…","[""3rd Person Perspective"", ""StashDB ID: f562975c-e209-464c-83ed-8ac18eb3a2e8"", ""Third Person Perspective""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5051…",47,0,0,15,0,0,0,"[{""7751""}]",[],1,0,"""f562975c-e209-464c-83ed-8ac18e…"
"""5053""","""4:3 Aspect Ratio""","""Footage shot in a 4:3 (1.33:1)…","[""1.33:1"", ""1.33:1 Aspect Ratio"", … ""StashDB ID: 6958c8ed-1948-46d2-89e0-cb48919bf8f1""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5053…",0,0,0,0,0,0,0,"[{""7749""}]",[],1,0,"""6958c8ed-1948-46d2-89e0-cb4891…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""7559""","""Young Man (22–30)""","""Male presented as generally yo…","[""StashDB ID: e7f1f848-4350-4bda-925c-b0123521b4de"", ""Young Guy"", … ""Youthful Man""]",false,"""2024-04-23T12:52:46Z""","""2025-01-26T10:33:27+02:00""",false,"""http://localhost:6969/tag/7559…",111,0,0,2,0,0,0,"[{""7750""}]",[],1,0,"""e7f1f848-4350-4bda-925c-b01235…"
"""7560""","""Young Woman (22–30)""","""Female character presented as …","[""Chick"", ""StashDB ID: 84ba8ef1-084c-46f8-b352-31154f5bfbbc"", … ""Youthful Woman""]",false,"""2024-04-23T12:52:46Z""","""2025-01-26T10:33:27+02:00""",false,"""http://localhost:6969/tag/7560…",1049,0,0,48,0,0,0,"[{""7750""}]",[],1,0,"""84ba8ef1-084c-46f8-b352-31154f…"
"""7563""","""Zentai""","""Skin-tight garment that covers…","[""StashDB ID: 6c0a2824-acd2-4b64-9a2c-634bd9e4d0d0"", ""Zentai Suit""]",false,"""2024-04-23T12:52:46Z""","""2025-01-26T10:33:27+02:00""",false,"""http://localhost:6969/tag/7563…",0,0,0,0,0,0,0,"[{""7762""}]",[],1,0,"""6c0a2824-acd2-4b64-9a2c-634bd9…"


In [196]:
# Merge df_stashdb_tags and df_stash_tags based on the 'name' column
merged_df = df_stashdb_tags.join(df_stash_tags, left_on='id', right_on='stashdb_id', how='full', suffix='_stash')

# Identify matching and non-matching tags
matching_tags = merged_df.filter(pl.col('id').is_not_null() & pl.col('id_stash').is_not_null())
stashdb_only_tags = merged_df.filter(pl.col('id_stash').is_null())
stash_only_tags = merged_df.filter(pl.col('id').is_null())

# Display results
print(f"Total matching tags: {len(matching_tags)}")
print(f"Tags only in StashDB: {len(stashdb_only_tags)}")
print(f"Tags only in Stash: {len(stash_only_tags)}")

merged_df

Total matching tags: 2557
Tags only in StashDB: 199
Tags only in Stash: 398


id,name,description,aliases,deleted,created,updated,category_id,category_name,category_description,category_group,id_stash,name_stash,description_stash,aliases_stash,ignore_auto_tag,created_at,updated_at,favorite,image_path,scene_count,scene_marker_count,image_count,gallery_count,performer_count,studio_count,group_count,parents,children,parent_count,child_count,stashdb_id
str,str,str,list[str],bool,str,str,str,str,str,str,str,str,str,list[str],bool,str,str,bool,str,i64,i64,i64,i64,i64,i64,i64,list[struct[1]],list[struct[1]],i64,i64,str
"""1257be8b-d1ec-4cb1-bb22-beeb89…","""2D Available""","""3D or VR scenes that offer a m…","[""2-D"", ""2D"", … ""Two-Dimensional""]",false,"""2020-05-01T09:37:09Z""","""2022-02-22T21:51:53Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE""","""5045""","""2D Available""","""3D or VR scenes that offer a m…","[""2-D"", ""2D"", … ""Two-Dimensional""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5045…",1,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""1257be8b-d1ec-4cb1-bb22-beeb89…"
"""52992c2c-4617-4540-8ca4-291e9c…","""3D Available""","""Offered in a format with a thr…","[""3-D"", ""3D"", … ""Three-Dimensional""]",false,"""2020-04-27T18:59:52Z""","""2022-02-22T21:52:15Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE""","""5049""","""3D Available""","""Offered in a format with a thr…","[""3-D"", ""3D"", … ""Three-Dimensional""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5049…",50,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""52992c2c-4617-4540-8ca4-291e9c…"
"""c3794d99-1b5b-47b3-86f7-75ff2d…","""3K Available""","""Scenes offered in a resolution…","[""1600p"", ""3K VP9"", … ""3KVR""]",false,"""2023-02-06T23:54:49.304855Z""","""2023-02-06T23:54:49.304855Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE""","""5050""","""3K Available""","""Scenes offered in a resolution…","[""1600p"", ""3K VP9"", … ""StashDB ID: c3794d99-1b5b-47b3-86f7-75ff2de748b8""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5050…",7,0,0,0,0,0,0,"[{""7752""}]",[],1,0,"""c3794d99-1b5b-47b3-86f7-75ff2d…"
"""f562975c-e209-464c-83ed-8ac18e…","""3rd Person Narrative""","""Features a storyline with fict…","[""3rd Person Perspective"", ""Third Person Perspective"", ""第三者撮り""]",false,"""2021-02-02T16:02:36Z""","""2024-12-01T16:57:51.952057Z""","""0319d5d6-a07f-4e0d-809d-c09fb1…","""Themes""","""Events, contexts, or fetishes …","""SCENE""","""5051""","""3rd Person Narrative""","""Features a storyline with fict…","[""3rd Person Perspective"", ""StashDB ID: f562975c-e209-464c-83ed-8ac18eb3a2e8"", ""Third Person Perspective""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5051…",47,0,0,15,0,0,0,"[{""7751""}]",[],1,0,"""f562975c-e209-464c-83ed-8ac18e…"
"""6958c8ed-1948-46d2-89e0-cb4891…","""4:3 Aspect Ratio""","""Footage shot in a 4:3 (1.33:1)…","[""1.33:1"", ""1.33:1 Aspect Ratio"", … ""Fullscreen""]",false,"""2022-08-08T00:33:32.647805Z""","""2022-08-26T06:14:19.530426Z""","""ef4ae6d1-d13c-4195-b47e-f245e4…","""Shot Type""","""Technical details of how a vid…","""SCENE""","""5053""","""4:3 Aspect Ratio""","""Footage shot in a 4:3 (1.33:1)…","[""1.33:1"", ""1.33:1 Aspect Ratio"", … ""StashDB ID: 6958c8ed-1948-46d2-89e0-cb48919bf8f1""]",false,"""2024-04-23T12:50:49Z""","""2025-01-26T10:32:59+02:00""",false,"""http://localhost:6969/tag/5053…",0,0,0,0,0,0,0,"[{""7749""}]",[],1,0,"""6958c8ed-1948-46d2-89e0-cb4891…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""283d982a-2399-46f3-a540-3707c7…","""Blowjob Sandwich""","""A visual ""sandwich"" effect dur…",[],false,"""2024-11-28T21:34:35.795903Z""","""2024-11-28T21:34:35.795903Z""","""feca7511-ac91-42c0-a032-8fb8f3…","""Acts""","""Various s

In [197]:
my_very_own_tags_parent_tag = stash.find_tag({ "name": "My Very Own Tags" })

df_stash_only_tags = df_stash_tags.filter(
    pl.col("id").is_in(stash_only_tags.select("id_stash").unique())
).filter(
    # Check if the tag doesn't have "My Very Own Tags" as parent
    pl.col("parents").map_elements(
        lambda parents: not any(parent.get('id') == my_very_own_tags_parent_tag['id'] for parent in parents),
        return_dtype=pl.Boolean
    )
).filter(
    ~pl.col("name").str.starts_with("Category:") & 
    ~pl.col("name").str.starts_with("Category Group:") & 
    ~pl.col("name").str.starts_with("AI_") & 
    ~pl.col("name").str.ends_with("_AI") &
    ~pl.col("name").str.starts_with("Data Quality Issue") & 
    ~pl.col("name").str.starts_with("Duplicate") & 
    ~pl.col("name").str.starts_with("Galleries") & 
    ~pl.col("name").str.starts_with("Group Makeup")
).select("id", "name", "aliases")
df_stash_only_tags

id,name,aliases
str,str,list[str]
"""7994""","""12K Available""","[""12K"", ""12K Shemale VR Porn"", … ""True 12K""]"
"""7995""","""Abandonment Play""",[]
"""7996""","""Aerial Cowgirl""","[""Keiran's Cowboy""]"
"""7997""","""African Accent""","[""Accent (African)""]"
"""7998""","""Against Glass""","[""Against Window"", ""Against the Glass"", … ""Woman Against Glass""]"
…,…,…
"""7967""","""Verified: Locations""",[]
"""8188""","""Virgin (Roleplay)""",[]
"""8189""","""Visible Marks""",[]


# Create category groups

In [175]:
# Get all unique category groups from StashDB tags
category_groups = df_stashdb_tags.select('category_group').drop_nulls().unique().to_series().to_list()

# Display the category groups
print("Unique category groups in StashDB:")
for group in sorted(category_groups):
    print(f"- {group}")
    tag_name = f"Category Group: {group}"
    existing_tag = stash.find_tag(tag_name)
    if existing_tag is None:
        stash.create_tag({
            "name": tag_name,
            "description": f"StashDB category group: {group}",
        })
        print(f"Created tag: {tag_name}")
    else:
        print(f"Tag already exists: {tag_name}")

Unique category groups in StashDB:
- ACTION
Tag already exists: Category Group: ACTION
- PEOPLE
Tag already exists: Category Group: PEOPLE
- SCENE
Tag already exists: Category Group: SCENE


# Create categories

In [176]:
# Get all unique categories from StashDB tags
unique_categories = df_stashdb_tags.select(['category_id', 'category_name', 'category_group', 'category_description']).drop_nulls().unique()

# Display the unique categories
print("Unique categories in StashDB:")
for category in unique_categories.iter_rows(named=True):
    print(f"- Name: {category['category_name'] or 'N/A'}")
    print(f"  ID: {category['category_id']}")
    print(f"  Group: {category['category_group'] or 'N/A'}")
    print(f"  Description: {category['category_description'] or 'N/A'}")
    print()

# Create tags for each unique category in Stash
for category in unique_categories.iter_rows(named=True):
    name = category['category_name']
    group = category['category_group']
    description = category['category_description']
    
    category_tag = stash.find_tag(f"Category: {name}")
    if category_tag is None:
        category_group_tag = stash.find_tag(f"Category Group: {group}")
        
        category_tag = stash.create_tag({
            "name": f"Category: {name}",
            "description": f"StashDB category: {name}",
            "parent_ids": [category_group_tag['id']] if category_group_tag else None,
        })
        print(f"Created category tag: {name}")
    else:
        aliases = ["StashDB ID: " + category['category_id']]
        stash.update_tag({ "id": category_tag['id'], "aliases": aliases })
        print(f"Updated category tag: {name}")


Unique categories in StashDB:
- Name: Misc
  ID: 7f4ddc1b-8169-4d5b-b764-04ad074c84a8
  Group: SCENE
  Description: Information about the video itself rather than its content, or various descriptors that do not fit other tag categories.

- Name: Breasts
  ID: be3adc0c-a3fe-40bf-98a7-5eacd9fb89fe
  Group: PEOPLE
  Description: Various descriptions of a performer's breasts or nipples, often regarding size.

- Name: Group Makeup
  ID: c456b6a1-75c4-41d1-8d87-08d7e062c7d8
  Group: SCENE
  Description: Number of scene partners and often their genders.

- Name: Height
  ID: 5c08369e-bbd9-4ea4-8c9f-081412408cd8
  Group: PEOPLE
  Description: Defined ranges of a performer's height, separated by gender.

- Name: Hair Style
  ID: ab179fb8-aa2f-4c75-86cf-0166dd8d8136
  Group: PEOPLE
  Description: Various descriptions of hair styles, cuts, textures, and lengths.

- Name: Locations
  ID: 33e17fdb-57df-42da-af42-42acad2245c6
  Group: SCENE
  Description: Where the action takes place.

- Name: Tatto

# Update descriptions

In [177]:
# Create records of tags that need updates
description_update_records = []

for row in df_stash_tags.iter_rows(named=True):
    stash_tag_name = row['name']
    stashdb_tag = df_stashdb_tags.filter(pl.col('name') == stash_tag_name)
    
    if not stashdb_tag.is_empty():
        stashdb_tag = stashdb_tag.to_dicts()[0]
        
        # Check if description needs updating
        if stashdb_tag['description'] != row['description']:
            description_update_records.append({
                'tag_id': row['id'],
                'name': stash_tag_name,
                'field': 'description',
                'current_value': row['description'] or '',  # Handle None values
                'proposed_value': stashdb_tag['description'] or '',
            })
        
df_description_updates = pl.DataFrame(description_update_records).sort(['name', 'field']).filter(pl.col('current_value') != pl.col('proposed_value'))
df_description_updates

tag_id,name,field,current_value,proposed_value
str,str,str,str,str
"""5161""","""Asian on White""","""description""","""""","""White women having sex with As…"
"""5776""","""English Language""","""description""","""""","""Scene shows performer(s) speak…"
"""6154""","""Hitchhiker""","""description""","""""","""Character asking strangers for…"
"""6581""","""No Underwear""","""description""","""A performer going without thei…","""A performer going without thei…"
"""7993""","""Standing Sex (DP)""","""description""","""""","""A female performer having vagi…"
"""7380""","""Train (Rail Transport)""","""description""","""At least some of the action ta…","""Some of the action takes place…"
"""7446""","""Unshaven""","""description""","""""","""Performers with hair on their …"
"""7562""","""Violet Wand""","""description""","""""","""Wand or rod designed to delive…"


In [178]:
for row in df_description_updates.iter_rows(named=True):
    print(row['name'])
    print(row['current_value'])
    print(row['proposed_value'])
    print()

    update_data = {
        "id": row['tag_id'],
        "description": row['proposed_value']
    }
    try:
        stash.update_tag(update_data)
        print(f"Updated tag: {row['name']}")
    except Exception as e:
        print(f"Error updating tag {row['name']}: {e}")

Asian on White

White women having sex with Asian Male(s)

Updated tag: Asian on White
English Language

Scene shows performer(s) speaking English.

Updated tag: English Language
Hitchhiker

Character asking strangers for a ride in their car

Updated tag: Hitchhiker
No Underwear
A performer going without their underwear, i.e without their panties.
A performer going without their underwear, i.e without their underpants or panties.

Updated tag: No Underwear
Standing Sex (DP)

A female performer having vaginal and anal penetrative sex simultaneously while standing up

Updated tag: Standing Sex (DP)
Train (Rail Transport)
At least some of the action takes place on a train or light rail vehicle.
Some of the action takes place on a train or light rail vehicle.

Updated tag: Train (Rail Transport)
Unshaven

Performers with hair on their genitals.

Updated tag: Unshaven
Violet Wand

Wand or rod designed to deliver a focused electrical stimulation to a performer.

Updated tag: Violet Wand


# Update aliases


In [179]:
# Create records of tags that need updates
alias_update_records = []

for row in df_stash_tags.iter_rows(named=True):
    stash_tag_name = row['name']
    stashdb_tag = df_stashdb_tags.filter(pl.col('name') == stash_tag_name)
    
    if not stashdb_tag.is_empty():
        stashdb_tag = stashdb_tag.to_dicts()[0]
        
        # Get current aliases and separate StashDB ID aliases
        current_aliases = set(row['aliases']) if row['aliases'] else set()
        current_stashdb_ids = {alias for alias in current_aliases 
                             if alias.startswith(stashdb_id_alias_prefix)}
        current_regular_aliases = current_aliases - current_stashdb_ids
        
        # Get proposed aliases from StashDB, excluding CJK
        proposed_aliases = {alias for alias in (stashdb_tag['aliases'] or []) 
                          if not contains_cjk(alias)}
        
        # Check if regular aliases need updating
        if current_regular_aliases != proposed_aliases:
            # Keep exactly one StashDB ID alias if it exists
            final_stashdb_id = next(iter(current_stashdb_ids)) if current_stashdb_ids else None
            
            # Combine proposed aliases with StashDB ID
            final_aliases = proposed_aliases
            if final_stashdb_id:
                final_aliases.add(final_stashdb_id)
            
            # Calculate differences for display
            to_add = proposed_aliases - current_regular_aliases
            to_remove = current_regular_aliases - proposed_aliases
            
            # Only proceed if there are changes
            if to_add or to_remove:
                # Format difference string
                diff_parts = []
                if to_add:
                    diff_parts.append(f"+ {', '.join(sorted(to_add))}")
                if to_remove:
                    diff_parts.append(f"- {', '.join(sorted(to_remove))}")
                
                alias_update_records.append({
                    'tag_id': row['id'],
                    'name': stash_tag_name,
                    'current_aliases': ', '.join(sorted(current_aliases)),
                    'proposed_aliases': ', '.join(sorted(final_aliases)),
                    'differences': ' | '.join(diff_parts),
                    'current_list': sorted(current_aliases),
                    'proposed_list': sorted(final_aliases)
                })

# Create DataFrame and sort by name
df_alias_updates = pl.DataFrame(alias_update_records).sort('name')

# Print summary
print(f"Found {len(df_alias_updates)} tags with non-CJK alias updates")
print("\nSample of proposed updates:")
print(df_alias_updates.select(['name', 'current_aliases', 'proposed_aliases', 'differences']).head())

df_alias_updates

Found 17 tags with non-CJK alias updates

Sample of proposed updates:
shape: (5, 4)
┌─────────────────────┬─────────────────────────┬─────────────────────────┬────────────────────────┐
│ name                ┆ current_aliases         ┆ proposed_aliases        ┆ differences            │
│ ---                 ┆ ---                     ┆ ---                     ┆ ---                    │
│ str                 ┆ str                     ┆ str                     ┆ str                    │
╞═════════════════════╪═════════════════════════╪═════════════════════════╪════════════════════════╡
│ Asian on White      ┆ StashDB ID:             ┆ AMWF, Asian Male White  ┆ + AMWF, Asian Male     │
│                     ┆ ca8f709f-14b2-4ba4…     ┆ Woman, …                ┆ White Woman…           │
│ Close Up Missionary ┆ Intimate Missionary,    ┆ Closeup Missionary,     ┆ + Closeup Missionary,  │
│                     ┆ StashDB I…              ┆ Intimate M…             ┆ StashDB …              │
│ Dry H

tag_id,name,current_aliases,proposed_aliases,differences,current_list,proposed_list
str,str,str,str,str,list[str],list[str]
"""5161""","""Asian on White""","""StashDB ID: ca8f709f-14b2-4ba4…","""AMWF, Asian Male White Woman, …","""+ AMWF, Asian Male White Woman…","[""StashDB ID: ca8f709f-14b2-4ba4-82d8-835efd4fe72a""]","[""AMWF"", ""Asian Male White Woman"", ""StashDB ID: ca8f709f-14b2-4ba4-82d8-835efd4fe72a""]"
"""5491""","""Close Up Missionary""","""Intimate Missionary, StashDB I…","""Closeup Missionary, Intimate M…","""+ Closeup Missionary, StashDB …","[""Intimate Missionary"", ""StashDB ID: d3c76c3a-05de-41be-b07d-d0084fd46f76""]","[""Closeup Missionary"", ""Intimate Missionary"", ""StashDB ID: d3c76c3a-05de-41be-b07d-d0084fd46f76""]"
"""5741""","""Dry Humping""","""Genital Rubbing, Outercourse, …","""Cock Rubbing Pussy, Genital Ru…","""+ Cock Rubbing Pussy, StashDB …","[""Genital Rubbing"", ""Outercourse"", ""StashDB ID: ed2bc198-c51c-42db-8c55-a930e1b4940e""]","[""Cock Rubbing Pussy"", ""Genital Rubbing"", … ""StashDB ID: ed2bc198-c51c-42db-8c55-a930e1b4940e""]"
"""5776""","""English Language""","""StashDB ID: 2262dfb9-fc07-4930…","""English, English Speaking, Eng…","""+ English, English Speaking, E…","[""StashDB ID: 2262dfb9-fc07-4930-83e3-0c8b680a8801""]","[""English"", ""English Speaking"", … ""StashDB ID: 2262dfb9-fc07-4930-83e3-0c8b680a8801""]"
"""6130""","""Headband""","""StashDB ID: bd857ec2-5993-43eb…","""Alice Band, Hair Band, Head Ba…","""+ Alice Band, Hair Band, Head …","[""StashDB ID: bd857ec2-5993-43eb-9a75-4a6d8c01ad3b""]","[""Alice Band"", ""Hair Band"", … ""StashDB ID: bd857ec2-5993-43eb-9a75-4a6d8c01ad3b""]"
…,…,…,…,…,…,…
"""7047""","""Sideboob""","""StashDB ID: 0bfc3c9a-3e69-4c46…","""Side Boob, Side-Boob, StashDB …","""+ Side Boob, Side-Boob, StashD…","[""StashDB ID: 0bfc3c9a-3e69-4c46-851c-c3009ee1029d""]","[""Side Boob"", ""Side-Boob"", ""StashDB ID: 0bfc3c9a-3e69-4c46-851c-c3009ee1029d""]"
"""7993""","""Standing Sex (DP)""","""StashDB ID: 01eb4084-62ac-498c…","""DP - Standing, Standing (DP), …","""+ DP - Standing, Standing (DP)…","[""StashDB ID: 01eb4084-62ac-498c-9115-43fd709c23ca""]","[""DP - Standing"", ""Standing (DP)"", … ""StashDB ID: 01eb4084-62ac-498c-9115-43fd709c23ca""]"
"""6620""","""Train (Oral Sex)""","""Daisychain, StashDB ID: 6ed47d…","""Daisy Chain, Daisychain, Oral …","""+ Daisy Chain, Oral Train, Sta…","[""Daisychain"", ""StashDB ID: 6ed47dea-8483-42f9-bb61-3e046e6e7fc1""]","[""Daisy Chain"", ""Daisychain"", … ""StashDB ID: 6ed47dea-8483-42f9-bb61-3e046e6e7fc1""]"


In [180]:
for row in df_alias_updates.iter_rows(named=True):
    update_data = {
        "id": row['tag_id'],
        "aliases": row['proposed_list']
    }
    try:
        stash.update_tag(update_data)
        print(f"Updated tag: {row['name']}")
    except Exception as e:
        print(f"Error updating tag {row['name']}: {e}")

Updated tag: Asian on White
Updated tag: Close Up Missionary
Updated tag: Dry Humping
Updated tag: English Language
Updated tag: Headband
Updated tag: Hitchhiker
Updated tag: Kissing - POV
Updated tag: Legs Up Missionary
Updated tag: Missionary
Updated tag: Mixed - POV
Updated tag: No Underwear
Updated tag: Pussy Rubbing
Updated tag: Sideboob
Updated tag: Standing Sex (DP)
Updated tag: Train (Oral Sex)
Updated tag: Unshaven
Updated tag: Violet Wand


# Clean out the CJK aliases from existing tags

In [181]:
# First add a column with cleaned aliases
df_stash_tags = df_stash_tags.with_columns(
    pl.col('aliases').map_elements(lambda x: [alias for alias in x if not contains_cjk(alias)], return_dtype=pl.List(pl.Utf8)).alias('cleaned_aliases')
)

# Find tags where current aliases differ from cleaned aliases
tags_to_update = df_stash_tags.filter(pl.col('aliases') != pl.col('cleaned_aliases'))

print(f"Found {len(tags_to_update)} tags with CJK aliases to remove")
print("\nSample of changes to make:")
print(tags_to_update.select([
    'name',
    'aliases',
    'cleaned_aliases'
]).head())

# Optional: Apply the updates
def apply_alias_cleanup(tags_df):
    for row in tags_df.iter_rows(named=True):
        update_data = {
            'id': row['id'],
            'aliases': row['cleaned_aliases']
        }
        
        try:
            stash.update_tag(update_data)
            print(f"Updated aliases for {row['name']}")
        except Exception as e:
            print(f"Error updating {row['name']}: {e}")

# Uncomment to apply the updates:
apply_alias_cleanup(tags_to_update)

tags_to_update_for_review = tags_to_update.select(['name', 'aliases', 'cleaned_aliases'])
tags_to_update_for_review


Found 0 tags with CJK aliases to remove

Sample of changes to make:
shape: (0, 3)
┌──────┬───────────┬─────────────────┐
│ name ┆ aliases   ┆ cleaned_aliases │
│ ---  ┆ ---       ┆ ---             │
│ str  ┆ list[str] ┆ list[str]       │
╞══════╪═══════════╪═════════════════╡
└──────┴───────────┴─────────────────┘


name,aliases,cleaned_aliases
str,list[str],list[str]


# Create new tags

In [200]:
stashdb_only_tags = df_stashdb_tags.filter(~pl.col('id').is_in(df_stash_tags.select('stashdb_id').to_series()))
stashdb_only_tags


id,name,description,aliases,deleted,created,updated,category_id,category_name,category_description,category_group
str,str,str,list[str],bool,str,str,str,str,str,str
"""42d9e5c4-1a1d-4c93-bf47-9086f2…","""12K Available""","""Scenes offered in a resolution…","[""12K"", ""12K Shemale VR Porn"", … ""True 12K""]",false,"""2024-12-03T05:31:48.278753Z""","""2024-12-03T05:31:48.278753Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE"""
"""d9804601-fabb-4d22-b996-a284b0…","""Abandonment Play""","""When a submissive is left on t…",[],false,"""2024-10-30T06:31:26.233834Z""","""2024-11-22T21:50:50.484026Z""","""0319d5d6-a07f-4e0d-809d-c09fb1…","""Themes""","""Events, contexts, or fetishes …","""SCENE"""
"""a98c64b8-0564-44dc-b19e-582751…","""Aerial Cowgirl""","""Cowgirl position with receiver…","[""Keiran's Cowboy""]",false,"""2021-02-23T22:13:27Z""","""2025-01-20T11:59:34.064585Z""","""feca7511-ac91-42c0-a032-8fb8f3…","""Acts""","""Various sexual acts or positio…","""ACTION"""
"""59e97272-623d-4e14-8b33-36de55…","""African Accent""","""Speaks with an accent native t…","[""Accent (African)""]",false,"""2025-01-22T04:08:55.722847Z""","""2025-01-22T04:08:55.722847Z""","""7f4ddc1b-8169-4d5b-b764-04ad07…","""Misc""","""Information about the video it…","""SCENE"""
"""6f2dde49-5c9a-495d-b58f-e20eab…","""Against Glass""","""Performer is pressed against g…","[""Against Window"", ""Against the Glass"", … ""Woman Against Glass""]",false,"""2024-11-23T20:34:58.67259Z""","""2024-11-23T20:34:58.67259Z""","""feca7511-ac91-42c0-a032-8fb8f3…","""Acts""","""Various sexual acts or positio…","""ACTION"""
…,…,…,…,…,…,…,…,…,…,…
"""9f24125b-8cb5-40b0-8826-6b6e7f…","""Unconscious""","""Features at least one characte…",[],false,"""2024-11-29T22:56:56.620834Z""","""2024-11-29T22:56:56.620834Z""","""0319d5d6-a07f-4e0d-809d-c09fb1…","""Themes""","""Events, contexts, or fetishes …","""SCENE"""
"""165ac407-9a7e-4603-a2ee-5b3e38…","""Vaginal Plug""","""Plug designed to be inserted v…","[""Plugged Pussy"", ""Pussy Plug"", ""Vaginally Plugged""]",false,"""2024-12-27T20:42:01.737553Z""","""2024-12-27T20:42:01.737553Z""","""bc5f4d70-c5f3-4cd6-bc60-d44700…","""Accessories""","""Notable devices or materials d…","""ACTION"""
"""7717e720-ee78-4ba0-a650-b2a9db…","""Virgin (Roleplay)""","""Performer who pretends to be a…",[],false,"""2024-10-24T05:32:32.795291Z""","""2024-10-24T05:32:32.795291Z""","""c423ad76-53f3-45a2-a865-87ee19…","""Roles""","""Common character archetypes or…","""SCENE"""


In [207]:
# Create tags in Stash which exist in StashDB but not in Stash
stashdb_only_tags = df_stashdb_tags.filter(~pl.col('id').is_in(df_stash_tags.select('stashdb_id').to_series()))

print(f"Number of tags in StashDB but not in Stash: {len(stashdb_only_tags)}")

new_tags = []
for stashdb_tag in stashdb_only_tags.iter_rows(named=True):
    # Check if the tag already exists in Stash
    existing_tag = stash.find_tag(stashdb_tag['name'])
    if existing_tag:
        # Check if the tag exists due to an alias
        if stashdb_tag['name'] in existing_tag['aliases']:
            print(f"Tag already exists due to alias: {stashdb_tag['name']}")
        else:
            print(f"Tag already exists: {stashdb_tag['name']}")
        continue
    
    # Find the category tag if it exists
    category_tag = None
    if stashdb_tag['category_name']:
        category_tag = stash.find_tag(f"Category: {stashdb_tag['category_name']}")
    
    # Prepare the tag data
    tag_data = {
        "name": stashdb_tag['name'],
        "description": stashdb_tag['description'],
    }
    
    # Add aliases if they exist
    if stashdb_tag['aliases']:
        tag_data["aliases"] = stashdb_tag['aliases'] + ["StashDB ID: " + stashdb_tag['id']]
    else:
        tag_data["aliases"] = ["StashDB ID: " + stashdb_tag['id']]
    
    # Add parent category if it exists
    if category_tag:
        tag_data["parent_ids"] = [category_tag['id']]
    
    new_tags.append(tag_data)

new_tags_df = pl.DataFrame(new_tags)
new_tags_df


Number of tags in StashDB but not in Stash: 199


name,description,aliases,parent_ids
str,str,list[str],list[str]
"""12K Available""","""Scenes offered in a resolution…","[""12K"", ""12K Shemale VR Porn"", … ""StashDB ID: 42d9e5c4-1a1d-4c93-bf47-9086f2016dcd""]","[""7752""]"
"""Abandonment Play""","""When a submissive is left on t…","[""StashDB ID: d9804601-fabb-4d22-b996-a284b00e8c5b""]","[""7751""]"
"""Aerial Cowgirl""","""Cowgirl position with receiver…","[""Keiran's Cowboy"", ""StashDB ID: a98c64b8-0564-44dc-b19e-5827517fed6c""]","[""7753""]"
"""African Accent""","""Speaks with an accent native t…","[""Accent (African)"", ""StashDB ID: 59e97272-623d-4e14-8b33-36de55ecc797""]","[""7752""]"
"""Against Glass""","""Performer is pressed against g…","[""Against Window"", ""Against the Glass"", … ""StashDB ID: 6f2dde49-5c9a-495d-b58f-e20eab8b5666""]","[""7753""]"
…,…,…,…
"""Unconscious""","""Features at least one characte…","[""StashDB ID: 9f24125b-8cb5-40b0-8826-6b6e7f049fe2""]","[""7751""]"
"""Vaginal Plug""","""Plug designed to be inserted v…","[""Plugged Pussy"", ""Pussy Plug"", … ""StashDB ID: 165ac407-9a7e-4603-a2ee-5b3e38a2aae2""]","[""7760""]"
"""Virgin (Roleplay)""","""Performer who pretends to be a…","[""StashDB ID: 7717e720-ee78-4ba0-a650-b2a9db2351dc""]","[""7756""]"


In [209]:
for tag in new_tags_df.iter_rows(named=True):
    # Create the tag in Stash
    try:
        new_tag = stash.create_tag(tag)
        print(f"Created tag: {new_tag['name']}")
    except Exception as e:
        print(f"Error creating tag: {e}")

print(f"Created {len(new_tags_df)} new tags in Stash.")


Created tag: 12K Available
Created tag: Abandonment Play
Created tag: Aerial Cowgirl
Created tag: African Accent
Created tag: Against Glass
Created tag: AI Model
Created tag: Anal Cowgirl - POV
Created tag: Anal Doggy Style - POV
Created tag: Anal Missionary - POV
Created tag: Anal Reverse Cowgirl - POV
Created tag: Asian Top
Created tag: Ass Smelling
Created tag: Baggy Shirt
Created tag: BBC Dildo
Created tag: Begging to Cum
Created tag: Big Penis Humiliation
Created tag: Black Top
Created tag: Blowjob Sandwich
Created tag: Blue Hair (Female)
Created tag: Bolt-Ons
Created tag: Boudoir
Created tag: Breast Fetish
Created tag: British Accent
Created tag: Bunched Up Skirt
Created tag: Cameraman
Created tag: Capture
Created tag: Chemical Play
Created tag: Chinese Accent
Created tag: Clavicle Piercing
Created tag: Cleave Gag
Created tag: Closed Missionary
Created tag: Clothes Removal
Created tag: Construction Site
Created tag: Coveralls
Created tag: Crawling
Created tag: Cruising
Created ta